## 1- introduction
* image registration: process of **transforming images into a common coordinate system** so **corresponding pixels represent homologous biological points**
* **Elastix**: open source **command-line program** for **intensity-based registration** of medical images, allows quickly configure, test and compare different registration methods
* **SimpleElastix**: an **extension of SimpleITK**, allows configuring and **running Elastix entirely in Python** and some other languages
* with SimpleElastix it's made easier and more memory and disk I/O efficient than Elastix

* image registration involves two images: fixed and moving
* transform: the spatial mapping of points from the fixed to points in moving (correspondence establishment)
* similarity metric: measure of how well fixed and moving image match
* optimizer: optimizes similarity metric over search space
* serach space: parameters of transform


* optimizer adjusts the parameters of transform in a way that minimizes the difference between the two images in terms of similarity metric
* so we just need to specify the metric we want to optimize
* when we want smooth deformations, we use regularization to penalize sharp transformation
* when we start with a high level of smoothing, and gradually sharpen the image, we use a multi-resolution approach

### registration components
#### image pyramids
* a multi-resolution pyramid strategy improves the capture range and robustness of registration
* three types of pyramid: SmoothingImagePyramid, RecursiveImagePyramid, ShrinkingImagePyramid
* **SmoothingImagePyramid** smoothes the image with Gaussian kernel at different scales
* **RecursiveImagePyramid** smoothes and downsamples the image
* **ShrinkingImagePyramid** merely downsamples the image


* parameters to set for multi-resolution strategy: **NumberOfResolutions** (in general 3 resolutions will be sufficient. if fixed and moving images are far away we migh want to increase it to 5 or 6, this way pays more attention to register large, dominant structures in the beginning) and **ImagePyramidSchedule**(defines amount of blurring and downsampling in each direction and for each resolution level) (if data is highly anisotropic, you might want to blur less in the direction of largest spacing).

example:
<br>(NumberOfResolutions 4)
<br>(FixedImagePyramidSchedule 8 8 8 4 4 4 2 2 2 1 1 1)

means that at resolution level 1, voxels are blurred with 4/2 voxels in each direction. (sigma is half pyramid schedule value)

#### masks
* if more interested in aligning substructures than glonal anatomy
* if you need an irregular region of interest (ROI), you can use masks
* when to use masks:
  * image contains artificial edges with no real meaning. the registration tries to align those and neglect the meaningful edges
  * image contains structures in the neighborhood of your ROI that may influence the registration within ROI
* only a fixed image mask is sufficient to focus the registrtion on the ROI, you only want to use mask for moving image, when the moving image contains highly perturbed grey levels near the ROI
* if using multi-resolution registration, set (ErodeMask "true"), since you do not want information from the artificial edge to flow into your ROI during smoothing step, (if the edges around the ROI are meaningful, set it to false), becuase edge will help to guide the registration

#### transforms
* constrains the solution space to that type of deformation (example: intra-subject applications: may be sufficient to consider only rigid transformation) (example2: a cross-sectional study demands more flexible transformation models to allow normal anatomical variabilities between patients)
* number of parameters correspond to the degree of freedom (DOF)
* DOF is equal to the dimensionality of the search space
* often good idea to start simple transforms and gradually increase complexity
* some common transforms: 
  * translation
  * rigid (rotation, translation)
  * Euler (rotaiton, translation)
  * affine (rotation, translation, scaling, shearing)
  * bspline (non-rigid)
  * Spline-Kernel transform (non-rigid)
  * weighted combination of any of these
* transform is from fixed image to moving image. this allows us to iterate over the fixed image and pick a pixel from the moving image for every pixel in the fixed image

#### metric
* similarity metric: measures degree of similarity between moving and fixed
* metric samples intensity values from fixed and transformed moving and evaluates the fitness value and derivatives (to pass to optimizer)
* some metrics can not handle inter-modality comparisons. some metrics have large capture range while others require initialization close to the optimil position
* you might require trial and error to find the best metric for a given problem
* **Mean Squared Difference (SSD)** pixel-wise intensity differences between two images squared mean. (good for the images with the same intensity distribution, like the same modality)
* **Normalized Correlation Coefficient (NCC)** pixel-wise cross-correlation normalized by the square root of the autocorrelation of the images. it's invariant to linear differences between intensity distributions. well for intra-modal CT registration.
* **Mutual Information (MI)** well-suited for multi-modal image pairs as well as mono-modal. sometimes better performance with normalized MI.
* **Mattes Mutual Information** samples te same pixels in every iteration.
* **Kappa Similarity Meric** measures the overlap of the segmented structures, specifically to register segmentations. (but usually better to conver a binary image to a distance map and apply one of the usual metrics.

#### optimizers
* estimates the optimal transform parameters in iterative fashion
* Gradient Deccent (GD), Robbins-Mobroe (RM), **Adaptive stochastic gradient descent(ASGD)**, conjugate gradient (CG), Quasi-Newton, simultaneous pertubation (SP)

#### samplers
* samples some points for the metric to evaluate only a subset of randomly sampled voxels
* using ImageSampler parameter (grid, random, random coordinate, full sampling)

#### interpolators
* is required to evaluate the image intensity at the mapped off-grid position
* NearestNeighborINterpolator: value of the spatially closest voxel. good for binary images
* LinearInterpolator: weighted average of surrounding voxels. (best time performance with this together with random coordinate sampler during optimization)
* BSplineInterpolatro: using b-spline approximations of user-defined order N. 1st order bspline correspond to linear interpolation. FinalBSplineInterpolatorFloat

#### images
* geometrical concepts associated with ITK images: ![Alt](images/se_001.JPG "Title")
* pixels represent the center of pixels

## 2- Hello World
* example ellustrating how to use SimpleElastix
* keeping it simple for now: a single function call we can specify fixed, moving image and type of registration. SimpleElatix then registers our images using sensible default parameters

### Registration With Translation Transform
consider these two brain MRIs.
![Alt](images/se_002.JPG "Title")
We identify objects are related by a simple spatial shift so a translation transform should suffice to align them (correct misalignment)

#### procedural interface
* short-hand notation (procedural or functional interface)
* less flexible than the object-oriented interface
* very simple to use

In [ ]:
# procedural interface
import SimpleITK as sitk

resultImage = sitk.Elastix(sitk.ReadImage("fixedImage.nii"), sitk.ReadImage("movingImage.nii"), "translation")


#### object-oriented interface
* code is more flexible but less simple
  * for example the final deformation field can be retrieved
  (like warping other images with the same transformation)
  * image quality is reduced from resampling twice the resulting image
* suitable for more advanced use cases and scripting purposes
* more verbose, but a lot more powerful

In [ ]:
import SimpleITK as sitk

fixedImage = sitk.ReadImage('fixedImage.nii')
movingImage = sitk.ReadImage('movingImage.nii')
parameterMap = sitk.GetDefaultParameterMap('translation')

elastixImageFilter = sitk.ElastixImageFilter()
elastixImageFilter.SetFixedImage(fixedImage)
elastixImageFilter.SetMovingImage(movingImage)
elastixImageFilter.SetParameterMap(parameterMap)
elastixImageFilter.Execute()

resultImage = elastixImageFilter.GetResultImage()
transformParameterMap = elastixImageFilter.GetTransformParameterMap()

* we can now transform an entire population of images (e.g. binary label images for segmentation of different brain regions) using the same parameter map and a single instance of transformix

In [ ]:
transformixImageFilter = sitk.TransformixImageFilter()
transformixImageFilter.SetTransformParameterMap(transformParameterMap)

population = ['image1.hdr', 'image2.hdr', ..., 'imageN.hdr']

for filename in population:
    transformixImageFilter.SetMovingImage(sitk.ReadImage(filename))
    transformixImageFilter.Execute()
    sitk.WriteImage(transformixImageFilter.GetResultImage(), "result_"+filename)

* the object-oriented interface facilitates reuse of components and dramatically simplifies book-keeping and boilerplate code
* next section: closer look at the parameter map interface that configures the registration components